In [2]:
import pandas as pd
import pandas_datareader.data as pdr
from pandas import Series, DataFrame
import numpy as np

import datetime
from datetime import date, timedelta
import yfinance as yf
yf.pdr_override()

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def fullPrint(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)   


In [3]:
df = pdr.get_data_yahoo("GOOG", start="2016-01-01",end="2020-12-31")
sp = "^GSPC"
spdf = pdr.get_data_yahoo(sp, start="2016-01-01",end="2020-12-31")
sortLogic = {
    'Close': 'last',
    'Volume': 'sum'
}
dfSorted = df.resample("W-FRI").agg(sortLogic)
spDfSorted = spdf.resample("W-FRI").agg(sortLogic)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
dfSorted['30WMA'] = dfSorted['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)

In [7]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['Close'],name='Price'),secondary_y=True)
fig.add_trace(go.Scatter(x=dfSorted.index, y=dfSorted['30WMA'],name="30WMA"),secondary_y=True)
fig.add_trace(go.Bar(x=dfSorted.index, y=dfSorted['Volume'],name="Volume"),secondary_y=False)
fig.show()

In [41]:
deltaX = 10.4
dfSorted['30WMASlope'] = dfSorted['30WMA'].diff()*500/(dfSorted['Close'].rolling(window=104).max()-dfSorted['Close'].rolling(window=104).min())/deltaX
fullPrint(dfSorted)

                  Close    Volume        30WMA  30WMASlope
Date                                                      
2016-01-08   714.469971  12585100          NaN         NaN
2016-01-15   694.450012  12450700          NaN         NaN
2016-01-22   725.250000  10137100          NaN         NaN
2016-01-29   742.950012  11388300          NaN         NaN
2016-02-05   683.570007  27932700          NaN         NaN
2016-02-12   682.400024  15659700          NaN         NaN
2016-02-19   700.909973   8485100          NaN         NaN
2016-02-26   705.070007   9808400          NaN         NaN
2016-03-04   710.890015  10191600          NaN         NaN
2016-03-11   726.820007  11287200          NaN         NaN
2016-03-18   737.599976   9905200          NaN         NaN
2016-03-25   735.299988   6133200          NaN         NaN
2016-04-01   749.909973   8283000          NaN         NaN
2016-04-08   739.150024   6064200          NaN         NaN
2016-04-15   759.000000   7224800          NaN         N

In [9]:
dfSorted['Percent'] = dfSorted['Close'].pct_change()
dfSorted['VolumePerc'] = dfSorted['Volume'].pct_change() 
spDfSorted['Percent'] = spDfSorted['Close'].pct_change()
dfSorted['RS'] = dfSorted['Percent'] - spDfSorted['Percent']
dfSorted

,Close,Volume,30WMA,Percent,VolumePerc,RS
Date,,,,,,
2016-01-08,714.469971,12585100,NaN,NaN,NaN,NaN
2016-01-15,694.450012,12450700,NaN,-0.028021,-0.010679,-0.006325
2016-01-22,725.250000,10137100,NaN,0.044352,-0.185821,0.030221
2016-01-29,742.950012,11388300,NaN,0.024405,0.123428,0.006922
2016-02-05,683.570007,27932700,NaN,-0.079925,1.452754,-0.048903
...,...,...,...,...,...,...
2020-12-04,1827.989990,7388200,1603.562425,0.019407,0.302965,0.002704
2020-12-11,1781.770020,6508300,1618.887157,-0.025285,-0.119095,-0.015644
2020-12-18,1731.010010,10237100,1630.058384,-0.028489,0.572930,-0.041031
